##### <font color='pink'> *Rozpoznawanie obrazów (2025)* </font>


## LAB 3: Klasyfikatory: Gaussian Naive Bayes, LDA, QDA, KNN, SVM

<font color='orange'> Prowadząca: dr inż. Urszula Libal </font>

 ---

# 1) Dane gaussowskie

ZAD.1. Wygeneruj 100 punktów z dwuwymiarowego rozkładu normalnego, a następnie je zwizualizuj.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generowanie danych
np.random.seed(42)  # Ustalony seed dla powtarzalności

# Klasa 1
mean1 = [1, 2]
cov1 = [[1, 0], [0, 3]]
points1 = np.random.multivariate_normal(mean1, cov1, 100)

# Klasa 2
mean2 = [-1, -1]
cov2 = [[4, 0.2], [0.2, 1]]
points2 = np.random.multivariate_normal(mean2, cov2, 100)

# Łączenie danych i etykiet klas
dane = np.vstack((points1, points2))  # Połączenie macierzy
klasa = np.hstack((np.ones(100), np.zeros(100)))  # Klasa 1 -> 1, Klasa 2 -> 0


# Rysowanie punktów
plt.scatter(dane[klasa == 1, 0], dane[klasa == 1, 1], color='red', label='Klasa 1')
plt.scatter(dane[klasa == 0, 0], dane[klasa == 0, 1], color='blue', label='Klasa 2')
plt.xlabel("Cecha 1")
plt.ylabel("Cecha 2")
plt.legend()
plt.grid(True)
plt.title("Wizualizacja punktów na płaszczyźnie")
plt.show()

ZAD.2. Poniższy kod dzieli dane na 90%:10% na zbiór treningowy i testowy, oraz przeprowadza klasyfikację za pomocą klasyfikatorów LDA, SVM i KNN (K=5).
- Sprawrawdź działanie algorytmów: porównaj dokładności klasyfikacji, macierze pomyłek i kształt obszarów decyzyjnych.
- Dopisz kod umożliwiający klasyfikację za pomocą NaiveBayes oraz QDA.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# Podział na zbiór uczący (90%) i testowy (10%)
X_train, X_test, y_train, y_test = train_test_split(dane, klasa, test_size=0.1, random_state=42)

# Klasyfikatory
classifiers = {
    "LDA": LDA(),
    "SVM": SVC(kernel='linear', probability=True),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Trenowanie i testowanie
results = {}
for name, model in classifiers.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    results[name] = {"accuracy": acc, "conf_matrix": cm}
    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} Confusion Matrix:\n{cm}\n")

# Wizualizacja obszarów decyzyjnych
def plot_decision_boundary(model, X, y, title):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.scatter(X[y == 1, 0], X[y == 1, 1], color='blue', alpha=0.5, label='klasa 1')
    plt.scatter(X[y == 0, 0], X[y == 0, 1], color='red', alpha=0.5, label='klasa 2')
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.xlabel("Cecha 1")
    plt.ylabel("Cecha 2")
    plt.legend()
    plt.title(title)
    plt.show()

for name, model in classifiers.items():
    plot_decision_boundary(model, dane, klasa, f"Obszary decyzyjne - {name}")


ZAD.3. Wykorzystaj poprzedni kod i przeprowadź klasyfikację dla podziału na zbiór treningowy i testowy:
- 25% treningowy, 75% testowy
- 50% treningowy, 50% testowy
- 75% treningowy, 25% testowy

Co się zmieniło? Jak pojedyncza pomyłka wpłwa na wynik accuracy?

---

ZAD.4. Sprawdzenie, czy dane pochodzą z rozkładu normalnego. 
- Które z użytych klasyfikatorów zakładają, że dane w klasach pochodza z rozkładu normalnego?

In [ ]:
import matplotlib.pyplot as plt

# Podział danych na klasy
class_1 = dane[klasa == 1]  # Punkty klasy 1
class_2 = dane[klasa == 0]  # Punkty klasy 2

# Rysowanie histogramów dla cechy x1
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(class_1[:, 0], bins=15, alpha=0.7, color='blue', label='Klasa 1')
plt.hist(class_2[:, 0], bins=15, alpha=0.7, color='red', label='Klasa 2')
plt.xlabel("Cecha x1")
plt.ylabel("Liczność")
plt.title("Histogram cechy x1")
plt.legend()
plt.grid(True)

# Rysowanie histogramów dla cechy x2
plt.subplot(1, 2, 2)
plt.hist(class_1[:, 1], bins=15, alpha=0.7, color='blue', label='Klasa 1')
plt.hist(class_2[:, 1], bins=15, alpha=0.7, color='red', label='Klasa 2')
plt.xlabel("Cecha x2")
plt.ylabel("Liczność")
plt.title("Histogram cechy x2")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


---
---

# 2) Dane Iris

ZAD. 6. Sprawdź, czy dane Iris w poszczególnych klasach pochodzą z rozkładu normalnego:
- Sprawdź wszystkie 4 cechy.
- Dobierz odpowiednią liczbę przedziałów histogramów (bins).

In [ ]:
# ZAD.6  Histogramy poszczególnych cech w klasach
# Cecha: 'sepal length (cm)'

import pandas as pd
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

sns.histplot(x='sepal length (cm)', data=iris_df, kde=True, hue=iris.target)
plt.title('sepal length (cm)')
plt.show()

# ZAD. Wyrysuj histogramy dla pozostałych cech
# Czy możemy założyć, że cechy te mają rozkład normalny?

ZAD.7. Jeśli można założyć rozkład normalny w klasach, zastosuj Gaussian Naive Bayes.
- Zbadaj zależność dokładności klasyfikacji od wielkości zbioru uczącego i testowego.
- Jaka jest wielkość zbioru testowego w poszczególnych przypadkach?
- Wylicz macierz pomyłekdla skrajnych przypadków.
- Jaki procent danych powinien zostać umieszony w zbiorze testowym?

In [ ]:
# ZAD.7 Naive Bayes (zależność dokładności klasyfikacji od wielkości zbioru uczącego i testowego)

# Wyrysuj dokładności klasyfikacji (accuracy) 
# w zależności od wielkości zbioru uczącego (parametr train_size albo test_size).

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

X, y = load_iris(return_X_y=True)

accuracy = []
train_part = np.arange(0.05,0.95, 0.02)

for tp in train_part:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=tp, random_state=42)
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    accuracy.append(metrics.accuracy_score(y_test, y_pred))

import matplotlib.pyplot as plt
print('Train set size =',train_part)
print('Accuracy =', np.round(accuracy, 2))

plt.plot(train_part,accuracy,label='Gaussian Naive Bayes')
plt.axis([0, 1, 0.5, 1.05])
plt.xlabel('Train set size [%]')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()


ZAD.8. Przeprowadź analizę dokładności klasyfikacji w zależności od wielkości zbioru uczącego dla pozostałych klasyfikatorów, tzn. LDA, QDA, KNN, SVM. Wyrysuj wszystkie wykresy na jednym obrazku.

ZAD.9. Wyznacz obszary decyzyjne dla danych Iris dla klasyfikatorów: GNB, LDA, QDA, KNN, SVM.

In [ ]:
# ZAD.9 Obszary decyzyjne (dane Iris) dla dwóch pierszych cech

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.naive_bayes import GaussianNB

iris = load_iris()
feature_1, feature_2 = np.meshgrid(
    np.linspace(iris.data[:, 0].min(), iris.data[:, 0].max()),
    np.linspace(iris.data[:, 1].min(), iris.data[:, 1].max())
)
grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T
gnb = GaussianNB().fit(iris.data[:, :2], iris.target)
y_pred = np.reshape(gnb.predict(grid), feature_1.shape)
display = DecisionBoundaryDisplay(xx0=feature_1, xx1=feature_2, response=y_pred)
display.plot()
display.ax_.scatter(iris.data[:, 0], iris.data[:, 1], c=iris.target, edgecolor="black")
plt.title("Obszary decyzyjne dla GNB")
plt.show()

---
---

# 3) Dane Moons

ZAD.10. Przeprowadź klasyfikację dla danych Moons różnymi klasyfikatorami. 
- Wyrysuj obszary decyzyjne.
- Spróbuj użyć także metody Kernel SVM.

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.2, random_state=42)

#...

 ---